# LOCALIZACIONES DEFINIDAS
posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
importar librerias

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

# FUNCIONES DE LLAMADA A LA API

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
# Asignar todos los token a una variable
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

In [9]:
# Asignar una variable a la url
url = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
#pyjsonviewer.view_data(json_data = a)

In [11]:
# A esta función entra una url, unas coordenadas y lo que queremos que nos devuelva la API(query).
# Nos devuelve un lista de diccionarios de sitios con esa descripción cerca de las coordenadas que hemos puesto.
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response

In [12]:
# Esta función se mete dentro del la lista de diccionarios previamente creada y nos devuelve todas las variavbles 
# que queremos en una lista de listas. En este caso he escogido el nombre, el tipo de negocio, la latitud, la 
# longitud el país y la dirección 

def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    cc = []
    adress = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        cc.append(i["venue"]["location"]["cc"])
    for i in response:
        adress.append(i["venue"]["location"]["formattedAddress"][0])      
    response2 = [name , type_, cc, adress,  lat, lng ]

    return response2

In [13]:
# Esta función crea un dataframe y lo transpone cambiando las filas por columnas. Además le da un nombre específico
# a cada columna en base a lo que esa columna muestra.
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Country", 3:"Adress", 4:"Latitude", 5:"Longitude", })
    return df1

In [14]:
# Está función junta los valores de latitud y longitud en un punto para que geopandas lo entienda.
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [15]:
# Esta función le añade cosas a la función anterior para que la localización quede de una manera clara para el 
# ordenador
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [16]:
# Esta función junta todas las funciones anteriores y crea una única función para crear un dataframe claro.
def df_final(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["location"] = get_loc(d)
    df_terminado = d[["Name","Type","Country", "Adress","Latitude","Longitude","location"]]
    return df_terminado

In [17]:
df_final(url, london_loc, "vegan")

,Name,Type,Country,Adress,Latitude,Longitude,location
0,What The Pitta,Vegetarian / Vegan Restaurant,GB,"Unit 52, Boxpark",51.523516,-0.076104,"{'type': 'Point', 'coordinates': (51.523515649..."
1,Vegan Nights London,Vegetarian / Vegan Restaurant,GB,London,51.521603,-0.071401,"{'type': 'Point', 'coordinates': (51.521603, -..."
2,Redemption,Restaurant,GB,320 Old St,51.52663,-0.081919,"{'type': 'Point', 'coordinates': (51.526630326..."
3,Genesis,Vegetarian / Vegan Restaurant,GB,144 Commercial St,51.521659,-0.075889,"{'type': 'Point', 'coordinates': (51.521658885..."
4,Vegan Yes,Vegetarian / Vegan Restaurant,GB,64 Brick Ln,51.518734,-0.071483,"{'type': 'Point', 'coordinates': (51.518734, -..."
...,...,...,...,...,...,...,...
95,Yard Sale Pizza,Pizza Place,GB,184 Hackney Rd,51.530633,-0.072591,"{'type': 'Point', 'coordinates': (51.530632970..."
96,Mien Tay,Vietnamese Restaurant,GB,122 Kingsland Rd,51.53044,-0.07718,"{'type': 'Point', 'coordinates': (51.530440436..."
97,All Star Lanes,Bowling Alley,GB,95 Brick Ln,51.522026,-0.07183,"{'type': 'Point', 'coordinates': (51.522026307..."
98,The Tea Rooms,Café,GB,Brick Lane (across from alley with Cafe 1001),51.521289,-0.071562,"{'type': 'Point', 'coordinates': (51.521289176..."


# FUNCTIONS TO GET CONDITIONS OF A LOCATION

# SCHOOLS

In [18]:
# Esta función llama a la API para ver todos los colegios que hay cuando le pasas una ubicación, los colegios que
# que busca son los colegios dentro de la lista de colegios. Aparte, añade una columna llamada classification
# al dataframe para saber que son todo colegios.
Schools = ["Nursery School","Elementary School","Middle School","High School"]
def get_schools(location, Schools):
    Nursery_sch = df_final(url, location, Schools[0])
    Elementary_sch = df_final(url, location, Schools[1])
    Middle_sch = df_final(url, location, Schools[2])
    High_sch = df_final(url, location, Schools[3])
    all_schools_bad_index = pd.concat([Nursery_sch, Elementary_sch, Middle_sch, High_sch])
    all_schools = all_schools_bad_index.reset_index(drop=True)
    all_schools.insert(1, "Classification", "School")
    return all_schools

# DESIGNING COMPANIES

In [19]:
#Esta función llama a la API para ver todos los estudios de diseño que hay cuando le pasas una ubicación, aparte 
# tambien mete una columna nueva para aclarar que son estudios de diseño ya que la APi te clasifica lo que te
# devuelve con otras cosas.
def get_design_studio(location):
    DesignCo = df_final(url, location, "Design Studio")
    DesignCo.insert(1, "Classification", "Designing Studios")
    return DesignCo


# TECH STARTUPS

In [20]:
#Esta función llama a la API para ver todos las start ups tecnológicas que hay cuando le pasas una ubicación.
# También añade una columna de clasificación para definirlas como Technological start ups.
def get_tech_startups(location):
    TechStaUp = df_final(url, location, "Tech Startup")
    TechStaUp.insert(1, "Classification", "Technological Startups")
    return TechStaUp

    

# LEISURE

In [21]:
# A esta función le pasas una localización y la palabra leisure. Entonces te devuelve todas las discotecas y sitios
# de ocio cerca de la localización establecida. Además tambien añade una columna clasificando todos esos sitios como
# Leisure.

leisure = ["Nightclub", "Piano Bar", "Pub", "Irish Pub"]
def get_leisure(location, leisure):
    Disco = df_final(url, location, leisure[0])
    Piano_bar = df_final(url, location, leisure[1])
    Pub = df_final(url, location, leisure[2])
    irish_pub = df_final(url, location, leisure[3])
    all_leisure_bad_index = pd.concat([Disco, Piano_bar, Pub, irish_pub])
    leisure = all_leisure_bad_index.reset_index(drop=True)
    leisure.insert(1, "Classification", "Leisure Places")
    return leisure

# TRAVELLING STATIONS

In [22]:
# Esta función recibe una url y la palabra stations. Y te devuelve un dataframe con todas las estaciones de bus,
# de metro y de tren cerca de tu localización. También clasifica todas estas estaciones en una columna, llamda 
# etaciones.
stations = ["Metro Station", "Light Rail Station", "Bus Station"  ]
def get_closeby_stations(location, stations):
    Metro = df_final(url, location, stations[0])
    Rail_Station = df_final(url, location, stations[1])
    Bus_Station = df_final(url, location, stations[1])
    all_stations_bad_index = pd.concat([Metro, Rail_Station, Bus_Station])
    stations = all_stations_bad_index.reset_index(drop=True)
    stations.insert(1, "Classification", "Travelling Stations")
    return stations

# PARKS

In [23]:
# A esta función le pasas una localización y te devuelve todos los parque cercanos. Además tambien incluye una 
# nueva colmna que clasifica a todas las variables como parques
def get_parks(location):
    Parks = df_final(url, location, "Park")
    Parks.insert(1, "Classification", "Parks")
    return Parks

# DATAFRAME GLOBAL Y LIMPIO

In [24]:
# A esta función le pasas una localización y te devuelve la un dataframe global con todo lo previamente calculado 
#con varias columnas.
def df_FINAL(location):
    a = get_schools(location, Schools)
    b = get_design_studio(location)
    c = get_tech_startups(location)
    d = get_leisure(location, leisure)
    e = get_closeby_stations(location,stations)
    f = get_parks(location)
    global_dataframe_bad_index = pd.concat([a,b,c,d,e,f])
    global_df_final = global_dataframe_bad_index.reset_index(drop=True)
    return global_df_final

# LONDON

In [25]:
london_schools = get_schools(london_loc, Schools)
london_designing_companies = get_design_studio(london_loc)
london_tech_startups = get_tech_startups(london_loc)
london_leisure = get_leisure(london_loc, leisure)
london_travelling_stations = get_closeby_stations(london_loc, stations)
london_parks = get_parks(london_loc)
london_df_entero = df_FINAL(london_loc)

In [26]:
# Importar todas las funciones de londres a un csv y el dataframe entero.
london_schools.to_csv("Data/London/ln_schools.csv")
london_designing_companies.to_csv("Data/London/ln_desing.csv")
london_tech_startups.to_csv("Data/London/ln_tech_startups.csv")
london_leisure.to_csv("Data/London/ln_leisure.csv")
london_travelling_stations.to_csv("Data/London/ln_stations.csv")
london_parks.to_csv("Data/London/ln_parks.csv")
london_df_entero.to_csv("Data/London/ln_entero.csv")

In [27]:
london_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
101,Croud Marketing,Technological Startups,Tech Startup,GB,39 Tabernacle Street,51.52226,-0.082099,"{'type': 'Point', 'coordinates': (51.52226, -0..."
154,Nomad,Leisure Places,Nightclub,GB,58 Old St (Golden Ln),51.524213,-0.095577,"{'type': 'Point', 'coordinates': (51.524212947..."
239,Kings Arms,Leisure Places,Pub,GB,27 Wormwood St (London Wall),51.516301,-0.0829,"{'type': 'Point', 'coordinates': (51.516301301..."
265,The Hand & Shears,Leisure Places,Pub,GB,1 Middle St,51.519352,-0.098865,"{'type': 'Point', 'coordinates': (51.519352336..."
340,Beech Gardens,Parks,Park,GB,Beech St.,51.520346,-0.09609,"{'type': 'Point', 'coordinates': (51.520346147..."
112,Swoon HQ,Technological Startups,Tech Startup,GB,91 - 93 Great Eastern Street (62 Southwark Bri...,51.52584,-0.083384,"{'type': 'Point', 'coordinates': (51.52584, -0..."
114,81 Leonard Street,Technological Startups,Tech Startup,GB,81 (Leonard street),51.524748,-0.082605,"{'type': 'Point', 'coordinates': (51.524747706..."
349,Braham Street Park,Parks,Park,GB,Braham St,51.514501,-0.072091,"{'type': 'Point', 'coordinates': (51.514500769..."
165,Basing House,Leisure Places,Nightclub,GB,25 Kingsland Rd.,51.529008,-0.077853,"{'type': 'Point', 'coordinates': (51.529007507..."
105,Makeshift,Technological Startups,Tech Startup,GB,12 Mallow St,51.524785,-0.088573,"{'type': 'Point', 'coordinates': (51.524785129..."


# SINGAPORE

In [28]:
singapore_schools = get_schools(singapore_loc, Schools)
singapore_designing_companies = get_design_studio(singapore_loc)
singapore_tech_startups = get_tech_startups(singapore_loc)
singapore_leisure = get_leisure(singapore_loc, leisure)
singapore_travelling_stations = get_closeby_stations(singapore_loc, stations)
singapore_parks = get_parks(singapore_loc)
singapore_df_entero = df_FINAL(singapore_loc)

In [29]:
singapore_schools.to_csv("Data/Singapore/sg_schools.csv")
singapore_designing_companies.to_csv("Data/Singapore/sg_designco.csv")
singapore_tech_startups.to_csv("Data/Singapore/sg_techstar.csv")
singapore_leisure.to_csv("Data/Singapore/sg_leisure.csv")
singapore_travelling_stations.to_csv("Data/Singapore/sg_stations.csv")
singapore_parks.to_csv("Data/Singapore/sg_parks.csv")
singapore_df_entero.to_csv("Data/Singapore/sg_entero.csv")

In [30]:
singapore_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
286,Duxton Blue,Leisure Places,Pub,SG,35 Duxton Rd,1.277924,103.843112,"{'type': 'Point', 'coordinates': (1.2779235318..."
282,5footway.inn Project Boat Quay,Leisure Places,Hostel,SG,"76, Boat Quay",1.288174,103.849336,"{'type': 'Point', 'coordinates': (1.2881736678..."
131,Lunar,Leisure Places,Nightclub,SG,"#01-03 The Cannery (3 River Valley Road, Clark...",1.290448,103.845475,"{'type': 'Point', 'coordinates': (1.2904478660..."
173,QUEENS The Club,Leisure Places,Nightclub,SG,ESPLANADE,1.2896,103.856622,"{'type': 'Point', 'coordinates': (1.2896000532..."
179,Mango One Thai Disco,Leisure Places,Nightclub,SG,Singapore,1.291933,103.856208,"{'type': 'Point', 'coordinates': (1.2919334306..."
187,PONG Singapore,Leisure Places,Nightclub,SG,3B River Valley Road #01-17 The Foundry,1.291255,103.845795,"{'type': 'Point', 'coordinates': (1.2912547552..."
257,Le Noir,Leisure Places,Lounge,SG,3C River Valley Road (Clarke Quay),1.289882,103.846299,"{'type': 'Point', 'coordinates': (1.2898819712..."
322,Hong Lim Park,Parks,Park,SG,New Bridge Rd. (Upper Pickering St.),1.286682,103.846458,"{'type': 'Point', 'coordinates': (1.2866816363..."
184,Grand Hollywood Disco & Pub,Leisure Places,Nightclub,SG,"Merchants' Court, #03-02 (3A Clarke Quay)",1.28962,103.845187,"{'type': 'Point', 'coordinates': (1.2896196542..."
194,London Boat Quay,Leisure Places,Pub,SG,55 Boat Quay,1.287263,103.849634,"{'type': 'Point', 'coordinates': (1.2872628099..."


# SAN FRANCISCO

In [31]:
sanfrancisco_schools = get_schools(san_francisco_loc, Schools)
sanfrancisco_designing_companies = get_design_studio(san_francisco_loc)
sanfrancisco_tech_startups = get_tech_startups(san_francisco_loc)
sanfrancisco_leisure = get_leisure(san_francisco_loc, leisure)
sanfrancisco_travelling_stations = get_closeby_stations(san_francisco_loc, stations)
sanfrancisco_parks = get_parks(san_francisco_loc)
sanfrancisco_df_entero = df_FINAL(san_francisco_loc)

In [32]:
sanfrancisco_schools.to_csv("Data/SanFrancisco/sf_schools.csv")
sanfrancisco_designing_companies.to_csv("Data/SanFrancisco/sf_design.csv")
sanfrancisco_tech_startups.to_csv("Data/SanFrancisco/sf_techstartups.csv")
sanfrancisco_leisure.to_csv("Data/SanFrancisco/sf_leisure.csv")
sanfrancisco_travelling_stations.to_csv("Data/SanFrancisco/sf_travelling.csv")
sanfrancisco_parks.to_csv("Data/SanFrancisco/sf_parks.csv")
sanfrancisco_df_entero.to_csv("Data/SanFrancisco/sf_entero.csv")

In [33]:
sanfrancisco_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
401,Francisco Park,Parks,Park,US,"San Francisco, CA 94109",37.804417,-122.420988,"{'type': 'Point', 'coordinates': (37.804417, -..."
96,Atelier Gabriella Daher,Designing Studios,Design Studio,US,Green St. (Polk St.),37.797764,-122.422242,"{'type': 'Point', 'coordinates': (37.797763552..."
299,California Trolley Line,Travelling Stations,Light Rail Station,US,"San Francisco, CA 94108",37.792223,-122.40926,"{'type': 'Point', 'coordinates': (37.792222993..."
130,Imgur HQ v2,Technological Startups,Tech Startup,US,415 Jackson St Ste 300,37.796523,-122.402464,"{'type': 'Point', 'coordinates': (37.796523360..."
404,Powell Street Parklet,Parks,Park,US,100-200 blocks (O'Farrell),37.786528,-122.407942,"{'type': 'Point', 'coordinates': (37.786528, -..."
92,Jak Home,Designing Studios,Design Studio,US,2423 Polk St (Union),37.799116,-122.422654,"{'type': 'Point', 'coordinates': (37.799116, -..."
278,Denny's,Leisure Places,Breakfast Spot,US,495 Beach St (at Jones St),37.806953,-122.417061,"{'type': 'Point', 'coordinates': (37.8069526, ..."
180,Pocket Gems HQ,Technological Startups,Tech Startup,US,220 Montgomery St (Bush Street),37.791501,-122.401699,"{'type': 'Point', 'coordinates': (37.791501055..."
79,Beast,Designing Studios,Design Studio,US,500 Sansome St,37.795513,-122.401277,"{'type': 'Point', 'coordinates': (37.795512549..."
205,Slalom,Technological Startups,Tech Startup,US,100 Pine St Fl 25 (Front St),37.792555,-122.398882,"{'type': 'Point', 'coordinates': (37.792555452..."


In [34]:
sanfrancisco_df_entero.Type.unique()

array(['Daycare', 'Nursery School', 'Preschool', 'Elementary School',
       'Private School', 'Middle School', 'High School', 'Museum',
       'Design Studio', 'Advertising Agency', 'Jewelry Store',
       'IT Services', 'Print Shop', 'Tech Startup', 'Cocktail Bar',
       'Nightclub', 'Bar', 'Rock Club', 'Taco Place',
       'American Restaurant', 'Irish Pub', 'Piano Bar', 'Pub', 'Dive Bar',
       'Beer Bar', 'New American Restaurant', 'Brewery', 'Comedy Club',
       'Sports Bar', 'Gastropub', 'Hotel', 'Breakfast Spot',
       'Metro Station', 'Light Rail Station', 'Bus Line', 'Park',
       'Playground', 'Monument / Landmark', 'Pedestrian Plaza', 'Pool',
       'Building', 'Neighborhood'], dtype=object)

In [35]:
london_df_entero.Type.unique()

array(['Nursery School', 'Elementary School',
       'Paper / Office Supplies Store', 'Middle School', 'High School',
       'Design Studio', 'Furniture / Home Store', 'Office',
       'Tech Startup', 'Coworking Space', 'Nightclub', 'Coffee Shop',
       'Pub', 'Social Club', 'Rock Club', 'Piano Bar',
       'French Restaurant', 'Bar', 'Beer Bar', 'Irish Pub',
       'Metro Station', 'Light Rail Station', 'Cemetery', 'Park',
       'Church', 'Plaza', 'Garden', 'Scenic Lookout', 'Neighborhood',
       'Road'], dtype=object)

In [36]:
singapore_df_entero.Type.unique()

array(['Nursery School', 'Elementary School', 'Middle School',
       'High School', 'General College & University', 'Design Studio',
       'Tech Startup', 'Office', 'Nightclub', 'Gift Shop', 'Jazz Club',
       'Pub', 'Brewery', 'Bar', 'Dive Bar', 'Hotel Bar', 'Beer Bar',
       'Canal', 'Australian Restaurant', 'Lounge', 'Café', 'Sports Bar',
       'Gay Bar', 'Hostel', 'German Restaurant', 'Gourmet Shop', 'Hotel',
       'Irish Pub', 'Metro Station', 'Light Rail Station', 'Park',
       'Outdoor Sculpture', 'Other Great Outdoors', 'Plaza', 'Roof Deck',
       'Lighthouse', 'Fire Station', 'Government Building', 'Event Space',
       'Playground', 'Waterfront', 'Trail', 'Theme Park', 'Church',
       'Road', 'Monument / Landmark', 'Neighborhood', 'Historic Site'],
      dtype=object)